# Object detection

In [1]:
# !mim download mmdet --config mask-rcnn_swin-t-p4-w7_fpn_1x_coco --dest .
%load_ext autoreload
%autoreload 2

from PIL import Image
import torch

from mmdet.apis import init_detector #, inference_detector
from mmengine.config import Config

from split_utils import SplitSwinTransformer, SplitTwoStageDetector, TwoInputIdentity

config_file = '../obj_det/mask-rcnn_swin-t-p4-w7_fpn_1x_coco.py'
checkpoint_file = '../obj_det/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_20210902_120937-9d6b7cfa.pth'

model_init = init_detector(config_file, checkpoint_file, device='cpu')  # or device='cuda:0'
# result = inference_detector(model, 'demo/demo.jpg')

# from mmdet.registry import MODELS
# MODELS.get('SwinTransformer')
# MODELS.get('TwoStageDetector')

Loads checkpoint by local backend from path: ../obj_det/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_20210902_120937-9d6b7cfa.pth


In [2]:
cfg = Config.fromfile('/obj_det/mask-rcnn_swin-t-p4-w7_fpn_1x_coco.py')
model = SplitTwoStageDetector.create_from_instance_and_cfg(model_init, cfg, cut_point=1)

prp = model.data_preprocessor
model.data_preprocessor = TwoInputIdentity()

# model.backbone = SplitSwinTransformer.create_from_instance_and_cfg(model.backbone, cfg)

In [3]:
model2 = SplitTwoStageDetector.create_from_cfg_and_checkpoint(config_file, checkpoint_file)



Loads checkpoint by local backend from path: ../obj_det/mask_rcnn_swin-t-p4-w7_fpn_1x_coco_20210902_120937-9d6b7cfa.pth


In [4]:
import json
from mmengine.registry import RUNNERS
from mmdet.utils import get_test_pipeline_cfg

from mmcv.transforms import Compose

cfg = model_init.cfg.copy()
test_pipeline = get_test_pipeline_cfg(cfg)

test_pipeline = Compose(test_pipeline)

# data_ = dict(img_path='demo/demo.jpg', img_id=0)


cfg_tr = cfg.copy()
cfg_tr['work_dir'] = './logs'
cfg_tr['train_dataloader']['dataset']['data_root'] = '../data/coco/'
# print(list(cfg_tr.train_dataloader.keys()))
runner = RUNNERS.build(cfg_tr)
# print(cfg_tr.train_dataloader)
# with open('data/coco/annotations/instances_train2017.json', 'r') as file:
#     data_json = json.load(file)


# data_ = train_pipeline(data_json)

# data_['inputs'] = data_['inputs'].unsqueeze(0)
# data_['data_samples'] = [data_['data_samples']]

# print(train_pipeline)
# print(list(data_json.keys()))

03/01 09:17:35 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.5 (default, May 30 2022, 18:17:32) [GCC 7.5.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 92037769
    GPU 0: NVIDIA GeForce RTX 2080 Ti
    CUDA_HOME: /usr/local/cuda-10.1
    NVCC: Cuda compilation tools, release 10.1, V10.1.24
    GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
    PyTorch: 1.8.1+cu102
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.7.0 (Git Hash 7aed236906b1f7a05c0917e5257a1af05e9ff683)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10.2
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arc

/local-scratch2/aharell/CFM-Task-Models/.venv/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /local-scratch2/aharell/CFM-Task-Models/.venv/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZNK3c106IValue23reportToTensorTypeErrorEv
  warn(f"Failed to load image Python extension: {e}")


03/01 09:17:36 - mmengine - INFO - Config:
auto_scale_lr = dict(base_batch_size=16, enable=False)
backend_args = None
data_root = 'data/coco/'
dataset_type = 'CocoDataset'
default_hooks = dict(
    checkpoint=dict(interval=1, type='CheckpointHook'),
    logger=dict(interval=50, type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    sampler_seed=dict(type='DistSamplerSeedHook'),
    timer=dict(type='IterTimerHook'),
    visualization=dict(type='DetVisualizationHook'))
default_scope = 'mmdet'
env_cfg = dict(
    cudnn_benchmark=False,
    dist_cfg=dict(backend='nccl'),
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0))
load_from = None
log_level = 'INFO'
log_processor = dict(by_epoch=True, type='LogProcessor', window_size=50)
max_epochs = 12
model = dict(
    backbone=dict(
        attn_drop_rate=0.0,
        convert_weights=True,
        depths=[
            2,
            2,
            6,
            2,
        ],
        drop_path_rate=0.2,
        d

In [5]:


dl = runner.train_dataloader
print(model.cut_point)
for i,data in enumerate(dl):
    model.zero_grad()
    model.eval()
    # im['inputs'] = torch.stack(im['inputs'])
    # print(list(data_.keys()))
    # data_ = train_pipeline(im)
    # data_ = prp(data_, False)
    # data_['inputs'] = model.backbone.split_forward_v2(data_['inputs'], output_layer=model.cut_point-1)
    # data_['inputs'] = {
    #                "hw_shape": data_['inputs'][0],
    #                "outs": data_['inputs'][1]}
    data = model.feature_frontend(data)
    print(model.backend_loss(data))
    # losses = model.loss(data_['inputs'], data_['data_samples'])
    # loss, losses = model.parse_losses(losses)
    break


loading annotations into memory...
Done (t=12.75s)
creating index...
index created!
1
(tensor(0.7063, grad_fn=<AddBackward0>), OrderedDict([('loss', tensor(0.7063, grad_fn=<AddBackward0>)), ('loss_rpn_cls', tensor(0.0172, grad_fn=<AddBackward0>)), ('loss_rpn_bbox', tensor(0.0275, grad_fn=<AddBackward0>)), ('loss_cls', tensor(0.1790, grad_fn=<MeanBackward0>)), ('acc', tensor(92.3828)), ('loss_bbox', tensor(0.2093, grad_fn=<MeanBackward0>)), ('loss_mask', tensor(0.2733, grad_fn=<MeanBackward0>))]))


In [7]:
# data_ = prp(data_, False)
# data_['inputs'] = model.backbone.split_forward(data_['inputs'], output_layer=model.cut_point-1)
# data_['inputs'] = {"x": data_['inputs'][0],
                #    "hw_shape": data_['inputs'][1],
                #    "outs": data_['inputs'][2]}

# data_['inputs'] = [data_['inputs']]
# data_['data_samples'] = [data_['data_samples']]

# # forward the model
with torch.no_grad():
    results = model.test_step(data)[0]

In [ ]:
model.backbone.stages[0]

SwinBlockSequence(
  (blocks): ModuleList(
    (0): SwinBlock(
      (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (attn): ShiftWindowMSA(
        (w_msa): WindowMSA(
          (qkv): Linear(in_features=96, out_features=288, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=96, out_features=96, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
          (softmax): Softmax(dim=-1)
        )
        (drop): DropPath()
      )
      (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (ffn): FFN(
        (layers): Sequential(
          (0): Sequential(
            (0): Linear(in_features=96, out_features=384, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
          )
          (1): Linear(in_features=384, out_features=96, bias=True)
          (2): Dropout(p=0.0, inplace=False)
        )
        (dropout_layer): DropPath()
        (gamma